In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load 
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#Calculations for District Summary
#Calculating Number of Schools, Total Number of students, Total Budget, Average Math Score and Average reading score:

Number_schools = school_data_complete["school_name"].nunique()
Number_students = len(student_data["Student ID"].unique())
Total_budget = school_data["budget"].sum()
Average_math = student_data["math_score"].mean()
Average_reading = student_data["reading_score"].mean()

#Extracting data and counting using by condition of students passing math, student passing reading and students passing both
math_pass_count = student_data.loc[(student_data['math_score'] >= 70), :].count()
reading_pass_count = student_data.loc[(student_data['reading_score'] >= 70), :].count()
Overall_pass_count = student_data.loc[(student_data["reading_score"] >=70) &(student_data['math_score'] >= 70), :].count()

#Calculating % of students passing math, passing reading and passing both subjects and storing them in a variable
Mathpass_percent = (math_pass_count["math_score"]/Number_students)*100
readingpass_percent = (reading_pass_count["reading_score"]/Number_students)*100
Overallpass_percent = (Overall_pass_count["Student ID"]/Number_students)*100


#Creating a Data frame to store and view the District Summary
District_df = pd.DataFrame({"Total Schools":[Number_schools],"Total Students":[Number_students],"Total Budget":[Total_budget],"Average Math Score":[Average_math],
                           "Average Reading Score":[Average_reading], "% Passing Math":[Mathpass_percent],"% Passing Reading":[readingpass_percent], "% Overall Passing":[Overallpass_percent]})



In [3]:
#Formatting Total Budget and Total Students
District_df["Total Budget"] =District_df["Total Budget"].map("${:,.2f}".format)
District_df["Total Students"] =District_df["Total Students"].astype(int).map("{:,.0f}".format)

District_df

Total Schools Total Students    Total Budget  Average Math Score  \
0             15         39,170  $24,649,428.00           78.985371   

   Average Reading Score  % Passing Math  % Passing Reading  % Overall Passing  
0               81.87784       74.980853          85.805463          65.172326

## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [4]:
#Creating a simplified Data Frame on which Groupby could be run
simp_df = school_data_complete[["school_name","type","size","budget","math_score","reading_score"]]

#Calculating Budget per student and then storing it in the Dataframe
Perstudent_budget = school_data_complete["budget"]/school_data_complete["size"]
simp_df["Per Student Budget"] = Perstudent_budget

school_types = school_data.set_index(["school_name"])["type"]

#Creating a data frame which is grouped by school name
grouped_schools = simp_df.groupby(["school_name"])



C:\Users\yamee\AppData\Local\Temp\ipykernel_26436\3573509008.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simp_df["Per Student Budget"] = Perstudent_budget


In [5]:
#Creating a data frame which is grouped by school name
grouped_schools = simp_df.groupby(["school_name"])



#Extracting and storing values from grouped data frame for School Summary Data Frame using mean and count functions
Perschool_ct = grouped_schools["math_score"].count()
TotalBudget = grouped_schools["budget"].mean()
PSBudget = grouped_schools["Per Student Budget"].mean()
Av_mscore = grouped_schools["math_score"].mean()
Av_rscore = grouped_schools["reading_score"].mean()

In [6]:
#Calculating % Passing math by using a conditional statement and then count per group by school and then dividing count of 
#students passing per school by the total students per school
passed_math = pd.DataFrame(simp_df.loc[simp_df["math_score"] >=70,:])
g_mp_sch = passed_math.groupby(["school_name"])
passedm_count = g_mp_sch["math_score"].count()
Passed_mathpercent = (passedm_count/Perschool_ct)*100

#Calculating % Passing reading by using a conditional statement and then count per group by school and then dividing count of 
#students passing per school by the total students per school
passed_reading = pd.DataFrame(simp_df.loc[simp_df["reading_score"] >=70,:])
g_rp_sch = passed_reading.groupby(["school_name"])
passedr_count = g_rp_sch["reading_score"].count()
Passed_rpercent = (passedr_count/Perschool_ct)*100


#Calculating % Overall passing by using a conditional statement and then count per group by school and then dividing count of 
#students passing per school by the total students per school

passed_Overall = pd.DataFrame(simp_df.loc[(simp_df["reading_score"] >=70) & (simp_df["math_score"] >= 70),:])
g_Op_sch = passed_Overall.groupby(["school_name"])
passedO_count = g_Op_sch["reading_score"].count()
Passed_Opercent = (passedO_count/Perschool_ct)*100

In [7]:
#Creating a Summary Data Frame to store series and view data in a table
SummarySHL_df = pd.DataFrame({"School Type": school_types,"Total Students": Perschool_ct, "Total School Budget":TotalBudget,
                              "Per Student Budget": PSBudget, "Average Math Score": Av_mscore, "Average Reading Score": Av_rscore,
                              "% Passing Math":Passed_mathpercent, "% Passing Reading":Passed_rpercent, 
                              "% Overall Passing":Passed_Opercent}) 

SummarySHL_df = SummarySHL_df.rename_axis(None)



In [8]:
#Formatting Total School Budget 
SummarySHL_df["Total School Budget"] =SummarySHL_df["Total School Budget"].map("${:,.2f}".format)

SummarySHL_df

School Type  Total Students Total School Budget  \
Bailey High School       District            4976       $3,124,928.00   
Cabrera High School       Charter            1858       $1,081,356.00   
Figueroa High School     District            2949       $1,884,411.00   
Ford High School         District            2739       $1,763,916.00   
Griffin High School       Charter            1468         $917,500.00   
Hernandez High School    District            4635       $3,022,020.00   
Holden High School        Charter             427         $248,087.00   
Huang High School        District            2917       $1,910,635.00   
Johnson High School      District            4761       $3,094,650.00   
Pena High School          Charter             962         $585,858.00   
Rodriguez High School    District            3999       $2,547,363.00   
Shelton High School       Charter            1761       $1,056,600.00   
Thomas High School        Charter            1635       $1,043,130.00   
Wilson High School        Charter            2283       $1,319,574.00   
Wright High School        Charter            1800       $1,049,400.00   

                       Per Student Budget  Average Math Score  \
Bailey High School                  628.0           77.048432   
Cabrera High School                 582.0           83.061895   
Figueroa High School                639.0           76.711767   
Ford High School                    644.0           77.102592   
Griffin High School                 625.0           83.351499   
Hernandez High School               652.0           77.289752   
Holden High School                  581.0           83.803279   
Huang High School                   655.0           76.629414   
Johnson High School                 650.0           77.072464   
Pena High School                    609.0           83.839917   
Rodriguez High School               637.0           76.842711   
Shelton High School                 600.0           83.359455   
Thomas High School                  638.0           83.418349   
Wilson High School                  578.0           83.274201   
Wright High School                  583.0           83.682222   

                       Average Reading Score  % Passing Math  \
Bailey High School                 81.033963       66.680064   
Cabrera High School                83.975780       94.133477   
Figueroa High School               81.158020       65.988471   
Ford High School                   80.746258       68.309602   
Griffin High School                83.816757       93.392371   
Hernandez High School              80.934412       66.752967   
Holden High School                 83.814988       92.505855   
Huang High School                  81.182722       65.683922   
Johnson High School                80.966394       66.057551   
Pena High School                   84.044699       94.594595   
Rodriguez High School              80.744686       66.366592   
Shelton High School                83.725724       93.867121   
Thomas High School                 83.848930       93.272171   
Wilson High School                 83.989488       93.867718   
Wright High School                 83.955000       93.333333   

                       % Passing Reading  % Overall Passing  
Bailey High School             81.933280          54.642283  
Cabrera High School            97.039828          91.334769  
Figueroa High School           80.739234          53.204476  
Ford High School               79.299014          54.289887  
Griffin High School            97.138965          90.599455  
Hernandez High School          80.862999          53.527508  
Holden High School             96.252927          89.227166  
Huang High School              81.316421          53.513884  
Johnson High School            81.222432          53.539172  
Pena High School               95.945946          90.540541  
Rodriguez High School          80.220055          52.988247  
Shelton High School            95.854628   

## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [9]:
#Using Sort function on Schools Summary Table created above to sort schools to get Top Performing Schools 
#based on "% Overall Passing"

TopFive_df = SummarySHL_df.sort_values(["% Overall Passing"], ascending=False)
TopFive_df.head()

School Type  Total Students Total School Budget  \
Cabrera High School     Charter            1858       $1,081,356.00   
Thomas High School      Charter            1635       $1,043,130.00   
Griffin High School     Charter            1468         $917,500.00   
Wilson High School      Charter            2283       $1,319,574.00   
Pena High School        Charter             962         $585,858.00   

                     Per Student Budget  Average Math Score  \
Cabrera High School               582.0           83.061895   
Thomas High School                638.0           83.418349   
Griffin High School               625.0           83.351499   
Wilson High School                578.0           83.274201   
Pena High School                  609.0           83.839917   

                     Average Reading Score  % Passing Math  % Passing Reading  \
Cabrera High School              83.975780       94.133477          97.039828   
Thomas High School               83.848930       93.272171          97.308869   
Griffin High School              83.816757       93.392371          97.138965   
Wilson High School               83.989488       93.867718          96.539641   
Pena High School                 84.044699       94.594595          95.945946   

                     % Overall Passing  
Cabrera High School          91.334769  
Thomas High School           90.948012  
Griffin High School          90.599455  
Wilson High School           90.582567  
Pena High School             90.540541

## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [10]:
#Using Sort function on Schools Summary Table created above to sort schools to get Bottom Performing Schools 
#based on "% Overall Passing"

WorstFive_df = SummarySHL_df.sort_values(["% Overall Passing"], ascending=True)
WorstFive_df.head()

School Type  Total Students Total School Budget  \
Rodriguez High School    District            3999       $2,547,363.00   
Figueroa High School     District            2949       $1,884,411.00   
Huang High School        District            2917       $1,910,635.00   
Hernandez High School    District            4635       $3,022,020.00   
Johnson High School      District            4761       $3,094,650.00   

                       Per Student Budget  Average Math Score  \
Rodriguez High School               637.0           76.842711   
Figueroa High School                639.0           76.711767   
Huang High School                   655.0           76.629414   
Hernandez High School               652.0           77.289752   
Johnson High School                 650.0           77.072464   

                       Average Reading Score  % Passing Math  \
Rodriguez High School              80.744686       66.366592   
Figueroa High School               81.158020       65.988471   
Huang High School                  81.182722       65.683922   
Hernandez High School              80.934412       66.752967   
Johnson High School                80.966394       66.057551   

                       % Passing Reading  % Overall Passing  
Rodriguez High School          80.220055          52.988247  
Figueroa High School           80.739234          53.204476  
Huang High School              81.316421          53.513884  
Hernandez High School          80.862999          53.527508  
Johnson High School            81.222432          53.539172

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [11]:
#Extracting 9th grade data into a data frame, grouping data frame based on schools then taking out mean math score per school
Nineth_df = pd.DataFrame(school_data_complete.loc[(school_data_complete["grade"] == "9th")])
Nineth_grouped = Nineth_df.groupby(["school_name"])
Nineth_meanM = Nineth_grouped["math_score"].mean()

#Extracting 10th grade data into a data frame, grouping data frame based on schools then taking out mean math score per school
Tenth_df = pd.DataFrame(school_data_complete.loc[(school_data_complete["grade"] == "10th")])
Tenth_grouped = Tenth_df.groupby(["school_name"])
Tenth_meanM =Tenth_grouped["math_score"].mean()

#Extracting 10th grade data into a data frame, grouping data frame based on schools then taking out mean math score per school
eleventh_df = pd.DataFrame(school_data_complete.loc[(school_data_complete["grade"] == "11th")])
eleventh_grouped = eleventh_df.groupby(["school_name"])
eleventh_meanM =eleventh_grouped["math_score"].mean()

#Extracting 10th grade data into a data frame, grouping data frame based on schools then taking out math reading per school
Twlth_df = pd.DataFrame(school_data_complete.loc[(school_data_complete["grade"] == "12th")])
Twlth_grouped = Twlth_df.groupby(["school_name"])
Twlth_meanM =Twlth_grouped["math_score"].mean()

#Creating a new Data frame the stores and views average math score per school per class
MathScoresbyGrades_df = pd.DataFrame({"9th":Nineth_meanM, "10th":Tenth_meanM, "11th":eleventh_meanM, "12th":Twlth_meanM})
MathScoresbyGrades_df = MathScoresbyGrades_df.rename_axis(None)
MathScoresbyGrades_df

9th       10th       11th       12th
Bailey High School     77.083676  76.996772  77.515588  76.492218
Cabrera High School    83.094697  83.154506  82.765560  83.277487
Figueroa High School   76.403037  76.539974  76.884344  77.151369
Ford High School       77.361345  77.672316  76.918058  76.179963
Griffin High School    82.044010  84.229064  83.842105  83.356164
Hernandez High School  77.438495  77.337408  77.136029  77.186567
Holden High School     83.787402  83.429825  85.000000  82.855422
Huang High School      77.027251  75.908735  76.446602  77.225641
Johnson High School    77.187857  76.691117  77.491653  76.863248
Pena High School       83.625455  83.372000  84.328125  84.121547
Rodriguez High School  76.859966  76.612500  76.395626  77.690748
Shelton High School    83.420755  82.917411  83.383495  83.778976
Thomas High School     83.590022  83.087886  83.498795  83.497041
Wilson High School     83.085578  83.724422  83.195326  83.035794
Wright High School     83.264706  84.010288  83.836782  83.644986

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [12]:
#Extracting 9th grade data into a data frame, grouping data frame based on schools then taking out mean reading per school
Nineth_df = pd.DataFrame(school_data_complete.loc[(school_data_complete["grade"] == "9th")])
Nineth_grouped = Nineth_df.groupby(["school_name"])
Nineth_meanR = Nineth_grouped["reading_score"].mean()

#Extracting 10th grade data into a data frame, grouping data frame based on schools then taking out mean reading per school
Tenth_df = pd.DataFrame(school_data_complete.loc[(school_data_complete["grade"] == "10th")])
Tenth_grouped = Tenth_df.groupby(["school_name"])
Tenth_meanR =Tenth_grouped["reading_score"].mean()

#Extracting 10th grade data into a data frame, grouping data frame based on schools then taking out mean reading per school
eleventh_df = pd.DataFrame(school_data_complete.loc[(school_data_complete["grade"] == "11th")])
eleventh_grouped = eleventh_df.groupby(["school_name"])
eleventh_meanR =eleventh_grouped["reading_score"].mean()

#Extracting 10th grade data into a data frame, grouping data frame based on schools then taking out mean reading per school
Twlth_df = pd.DataFrame(school_data_complete.loc[(school_data_complete["grade"] == "12th")])
Twlth_grouped = Twlth_df.groupby(["school_name"])
Twlth_meanR =Twlth_grouped["reading_score"].mean()

#Creating a new Data frame the stores and views average reading score per school per class
ReadingScoresbyGrades_df = pd.DataFrame({"9th":Nineth_meanR, "10th":Tenth_meanR, "11th":eleventh_meanR, "12th":Twlth_meanR})
ReadingScoresbyGrades_df = ReadingScoresbyGrades_df.rename_axis(None)
ReadingScoresbyGrades_df

9th       10th       11th       12th
Bailey High School     81.303155  80.907183  80.945643  80.912451
Cabrera High School    83.676136  84.253219  83.788382  84.287958
Figueroa High School   81.198598  81.408912  80.640339  81.384863
Ford High School       80.632653  81.262712  80.403642  80.662338
Griffin High School    83.369193  83.706897  84.288089  84.013699
Hernandez High School  80.866860  80.660147  81.396140  80.857143
Holden High School     83.677165  83.324561  83.815534  84.698795
Huang High School      81.290284  81.512386  81.417476  80.305983
Johnson High School    81.260714  80.773431  80.616027  81.227564
Pena High School       83.807273  83.612000  84.335938  84.591160
Rodriguez High School  80.993127  80.629808  80.864811  80.376426
Shelton High School    84.122642  83.441964  84.373786  82.781671
Thomas High School     83.728850  84.254157  83.585542  83.831361
Wilson High School     83.939778  84.021452  83.764608  84.317673
Wright High School     83.833333  83.812757  84.156322  84.073171

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [13]:
#Creating four reasonable bins
bins = [0, 586, 631, 646, 681]

#Assigning bins group names
group_names = ["<$585","$585-630","$630-645","$645-680"]


In [14]:
#Creating a new data frame to apply bins in from Summary Data Frame
ForBins_df = SummarySHL_df[["School Type","Total Students","Per Student Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading", "% Overall Passing"]]


#Adding Bins classification in the Data Frame
ForBins_df["Spending Ranges (Per student)"] = pd.cut(ForBins_df["Per Student Budget"], bins, include_lowest=True, labels=group_names)

#Using groupby to get a Data Frame grouped according to bins and then storing and printing in form of a table

PerSR_df = ForBins_df.groupby("Spending Ranges (Per student)")

FinalbySpending_df = PerSR_df["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"].mean()

FinalbySpending_df.round(2)


C:\Users\yamee\AppData\Local\Temp\ipykernel_26436\4096953029.py:12: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  FinalbySpending_df = PerSR_df["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"].mean()


Average Math Score  Average Reading Score  \
Spending Ranges (Per student)                                              
<$585                                       83.46                  83.93   
$585-630                                    81.90                  83.16   
$630-645                                    78.52                  81.62   
$645-680                                    77.00                  81.03   

                               % Passing Math  % Passing Reading  \
Spending Ranges (Per student)                                      
<$585                                   93.46              96.61   
$585-630                                87.13              92.72   
$630-645                                73.48              84.39   
$645-680                                66.16              81.13   

                               % Overall Passing  
Spending Ranges (Per student)                     
<$585                                      90.37  
$585-630                                   81.42  
$630-645                                   62.86  
$645-680                                   53.53

## Scores by School Size

* Perform the same operations as above, based on school size.

In [15]:
#Creating four reasonable bins and assigning bins group names
bins_size = [0, 1001, 2001, 5001]
group_names_size = ["Small(<1000)","Medium(1000-2000)","Large(2000-5000)"]

In [16]:
#Adding Bins classification in the Data Frame
ForBins_df["School Size"] = pd.cut(ForBins_df["Total Students"], bins_size, include_lowest=True, labels=group_names_size)

#Using groupby to get a Data Frame grouped according to bins and then storing and printing in form of a table
PerSize_df = ForBins_df.groupby("School Size")


In [17]:
#Printing Table
PerSize_df["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"].mean()


C:\Users\yamee\AppData\Local\Temp\ipykernel_26436\486233272.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  PerSize_df["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"].mean()


Average Math Score  Average Reading Score  % Passing Math  \
School Size                                                                    
Small(<1000)                83.821598              83.929843       93.550225   
Medium(1000-2000)           83.374684              83.864438       93.599695   
Large(2000-5000)            77.746417              81.344493       69.963361   

                   % Passing Reading  % Overall Passing  
School Size                                              
Small(<1000)               96.099437          89.883853  
Medium(1000-2000)          96.790680          90.621535  
Large(2000-5000)           82.766634          58.286003

## Scores by School Type

* Perform the same operations as above, based on school type

In [18]:
#Since School type is already present in Data Frame applying groupby directly on School Summary Data Frame
Grouped_schtype = SummarySHL_df.groupby("School Type")

Grouped_schtype["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"].mean()

C:\Users\yamee\AppData\Local\Temp\ipykernel_26436\194147714.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  Grouped_schtype["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"].mean()


Average Math Score  Average Reading Score  % Passing Math  \
School Type                                                              
Charter               83.473852              83.896421       93.620830   
District              76.956733              80.966636       66.548453   

             % Passing Reading  % Overall Passing  
School Type                                        
Charter              96.586489          90.432244  
District             80.799062          53.672208

## Trend Analysis

- When Data is analysed based on Average Math and Reading Scores and percentages of students passing Math, Reading and Overall it can be noticed that Charter schools have significantly better mean scores and higher mean percentages of students passing Overall. If we look at top performing schools they are Charter Schools and the Bottom performing schools are all District Schools. We can say that students in Charter Schools perform better than students in District Schools.  

- It can also be noticed that when scores and rate of passing subjects & overall is viewed according to school size and budget per student there is no visable/clear trend. This indicates the size of school and amount of budget allocated per student does not really determine students'/school's performance. Infact the the bottom performing schools have significantly higher budgets than the high performing schools.

- However what I noticed is that each school has its own trend in each grade level. Every school has similar performance per subject for all grades (9th to 12th). For instance if we look at reading scores for Baily High School, all grades have a mean score falling within the range of (81% +/- 0.5%) same for Cabera High School (84% +/- 0.5%).
This is also same for math scores.
